In [43]:
import numpy as np
import pandas as pd
import random as rd
import seaborn as sns
import itertools as it
import matplotlib.pyplot as plt

In [44]:
caracteristiques = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/e22ba475-45a3-46ac-a0f7-9ca9ed1e283a', sep=';', na_values=[-1])
lieux = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/2ad65965-36a1-4452-9c08-61a6c874e3e6', sep=';', na_values=[-1])
vehicules = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/780cd335-5048-4bd6-a841-105b44eb2667', sep=';', na_values=[-1])
usagers = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/36b1b7b3-84b4-4901-9163-59ae8a9e3028', sep=';', na_values=[-1])

In [45]:
data = (usagers.merge(vehicules, on='id_vehicule', suffixes=(None, '_2'))
               .merge(caracteristiques, on='Num_Acc', suffixes=(None, '_3'))
               .merge(lieux, on='Num_Acc'))
data.head()

,Num_Acc,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,...,prof,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma
0,201900000001,138 306 524,B01,2,2,4,2,2002,0,1,...,1,6,900,2,NaN,NaN,1,2,1,70
1,201900000001,138 306 524,B01,1,1,4,2,1993,5,1,...,1,6,900,2,NaN,NaN,1,2,1,70
2,201900000001,138 306 525,A01,1,1,1,1,1959,0,1,...,1,6,900,2,NaN,NaN,1,2,1,70
3,201900000002,138 306 523,A01,1,1,4,2,1994,0,1,...,4,3,845,2,NaN,NaN,1,0,1,70
4,201900000003,138 306 520,A01,1,1,1,1,1996,0,1,...,1,10,500,3,NaN,NaN,1,0,1,90


On supprime tous les identifiants, sauf celui des usagers (plus l'année, qui est la même pour tous les accidents de la base)

In [46]:
data.drop(inplace=True, columns=['Num_Acc', 'id_vehicule', 'num_veh', 'Num_Acc_2', 'num_veh_2', 'an'])

On supprime également les variables avec trop de valeurs manquantes

In [47]:
data.drop(inplace=True, columns=['lartpc', 'larrout', 'occutc','v2' ,'v1' ,'voie' ,'adr', 'trajet', 'secu2', 'secu3', 'locp', 'actp', 'etatp'])

In [48]:
data.head()

,place,catu,grav,sexe,an_nais,secu1,senc,catv,obs,obsm,...,nbv,vosp,prof,pr,pr1,plan,surf,infra,situ,vma
0,2,2,4,2,2002,1,2,7,0,2,...,10,0,1,6,900,2,1,2,1,70
1,1,1,4,2,1993,1,2,7,0,2,...,10,0,1,6,900,2,1,2,1,70
2,1,1,1,1,1959,1,2,17,1,0,...,10,0,1,6,900,2,1,2,1,70
3,1,1,4,2,1994,1,1,7,4,0,...,2,0,4,3,845,2,1,0,1,70
4,1,1,1,1,1996,1,1,7,0,2,...,8,0,1,10,500,3,1,0,1,90


In [49]:
data.columns

Index(['place', 'catu', 'grav', 'sexe', 'an_nais', 'secu1', 'senc', 'catv',
       'obs', 'obsm', 'choc', 'manv', 'motor', 'jour', 'mois', 'hrmn', 'lum',
       'dep', 'com', 'agg', 'int', 'atm', 'col', 'lat', 'long', 'catr', 'circ',
       'nbv', 'vosp', 'prof', 'pr', 'pr1', 'plan', 'surf', 'infra', 'situ',
       'vma'],
      dtype='object')

On transforme les variables catégorielles en suite d'indicatrices :

In [50]:
categories = {
    'place': ['Avant gauche', 'Avant droite', "Passager arrière (à droite)", "Passager arrière (à gauche)", "Passager arrière (au milieu)", "Passager avant (au milieu)", "Passager central (à gauche)", "passager central (au mileu)", "Passager central (à droite)"],
    'catu': ['Conducteur', 'Passager', 'Piéton'],
    'grav': ['Indemne', 'Tué', 'Blessé hospitalisé', 'Blessé léger'],
    'sexe': ['Homme', 'Femme'],
    'secu1': ['Ceinture', 'Casque', 'Dispositif enfant', 'Gilet réflechissant', 'Airbag', 'Gants', 'Gants + Airbag'], # Il va falloir gérer le cas Gants+Airbag à la main
    'senc': ['Croissant', 'Décroissant'], #Pertinence ?
    'catv': ['Bicyclette', 'Cyclomoteur <50cm3', 'Voiturette', '_', '_', '_', 'VL seul', '_', '_', 'VU seul', '_', '_', 'PL seul entre 3,5T et 7,5T', 'PL seul > 7,5T', 'PL > 3,5T avec remorque', 'Tracteur routier seul', 'Tracteur routier + semi-remorque', '_', '_', 'Engin spécial', 'Tracteur agricole'] + 8*['_'] + ['Scooteur >50cm3', 'Motocyclette entre 50 et 125cm3', 'Scooter entre 50 et 125cm3', 'Motocyclette >125cm3', 'Scooter >125cm3', 'Quad léger', 'Quad lourd', 'Autobus', 'Autocar', 'Train', 'Tramway', '3RM <=50cm3', '3RM entre 50 et 125cm3', '3RM >125cm3'] + 6*['_'] + ['EPD à moteur'] + 9*['_'] + ['EPD sans moteur'] + 19*['_'] + ['VAE'],
    'obs': ['Obstacle : '+ obs for obs in ['Véhicule en stationnement', 'Arbre', 'Glissière métallique', 'Glissière béton', 'Autre glissière', 'Bâtiment, mur, pile de pont', 'Support de signalisation vertical ou poste d\'appel d\'urgence', 'Poteau', 'Mobilier urbain', 'Parapet', 'Ilot, refuge, borne haute', 'Bordure de trottoir', 'Fossé, talus, paroi rocheuse', 'Autre obstacle fixe sur chaussée', 'Autre obstacle fixe hors chaussée', "Pas d'obstacle", "Buse, tête d'aqueduc"]],
    'obsm': ["Piéton", "Véhicule", "Véhicule sur rail", "Animal domestique", "Animal sauvage", "Autre obstacle mobile"],
    'choc': ["Choc à l'avant", "Choc à l'avant droit", "Choc à l'avant gauche", "Choc à l'arrière", "Choc à l'arrère droit", "Choc à l'arrière gauche", "Choc côté droit", "Choc côté gauche", "Chocs multiples"],
    'manv': ["Sans changement de direction", "Même sens, même file", "Entre 2 files", "En marche érrière", "À contresens", "En franchissant le terre-plein", "Dans le couloir bus, même sens", "Dans le couloir bus, sens inverse", "En s'insérant", "En faisant demi-tour sur la chaussée", "En changeant de file à gauche", "En changeant de file à droite", "En se déportant à gauche", "En se déportant à droite", "En tournant à gauche", "En tournant à droite", "En dépassant à gauche", "En dépassant à droite", "En traversant la chaussée", "Pendant une manoeuvre de stationnement", "Pendant une manoeuvre d'évitement", "Pendant l'ouverture d'une porte", "À l'arrêt", "En stationnement", "Circulant sur le trottoir", "Autre manoeuvre"],
    'motor': ["Moteur à hydrocarbures", "Moteur hybride", "Moteur électrique", "Moteur à hydrogène", "Propulsion humaine", "Autre propulsion"],
    'lum': ["Plein jour", "Crépuscule ou aube", "Nuit sans éclairage public", "Nuit avec éclairage public éteint", "Nuit avec éclairage public allumé"],
    'agg': ["Hors agglomération", "En agglomération"],
    'int': ["Hors intersection", "Intersection en X", "Intersection en T", "Intersection en Y", "Intersection à plus de 4 branches", "Giratoire", "Place", "Passage à niveau", "Autre intersection"],
    'atm': ["Conditions atmosphériques normales", "Pluie légère", "Pluie forte", "Neige ou grêle", "Brouillard ou fumée", "Vent fort ou tempête", "Temps éblouissant", "Temps couvert", "Autres conditions atmosphériques"],
    'col': ["Collision frontale (2 véhicules)", "Collision par l'arrière (2 véhicules)", "Collision par le côté (2 véhicules)", "Collision en chaîne (3 véhicules ou plus)", "Collisions multiples (3 véhicules ou plus)", "Autre collision", "Sans collision"],
    'catr': ["Autoroute", "Route nationale", "Route départementale", "Voie communale", "Hors réseau public", "Parc de stationnement", "Routes de métropole urbaine", "Autre route"],
    'circ': ["Circulation à sens unique", "Circulation bidirectionnelle", "Circulation à chaussées séparées", "Circulation avec voies d'affectation variables"],
    'vosp': ["Présence d'une piste cyclable", "Présence d'une bande cyclable", "Présence d'une voie réservée"],
    'prof': ["Route plate", "Route en pente", "Sommet de côte", "Bas de côte"],
    'plan': ["En ligne droite", "En courbe à gauche", "En courbe à droite", "En 'S'"],
    'surf': ["Surface normale", "Surface mouillée", "Présence de flaques", "Chaussée inondée", "Chaussée enneigée", "Chaussée boueuse", "Chaussé verglacée", "Présence d'un corps gras sur la chaussée (huile...)", "Autre état de la chaussée"],
    'infra': ["Souterrain, tunnel", "Pont, autopont", "Bretelle d'échangeur ou de raccordement", "Voie ferrée", "Carrefour aménagé", "Zone piétonne", "Zone de péage", "Chantier", "Autre type d'infrastructure"],
    'situ': ["Sur chaussée", "Sur bande d'arrêt d'urgence", "Sur accotement", "Sur trottoir", "Sur piste cyclable", "Sur autre voie spéciale", "Autre lieu"]
}

In [51]:
for var, labels in categories.items():
    for i, label in enumerate(labels):
        if label != '_':
            data[label] = (data[var]==i+1).astype(int)
    data.drop(inplace=True, columns=[var])
data['Gants'] = data['Gants + Airbag']
data['Airbag'] = data['Gants + Airbag']
data.drop(inplace=True, columns=['Gants + Airbag'])
data.head()

,an_nais,jour,mois,hrmn,dep,com,lat,long,nbv,pr,...,Zone de péage,Chantier,Autre type d'infrastructure,Sur chaussée,Sur bande d'arrêt d'urgence,Sur accotement,Sur trottoir,Sur piste cyclable,Sur autre voie spéciale,Autre lieu
0,2002,30,11,01:30,93,93053,"48,8962100","2,4701200",10,6,...,0,0,0,1,0,0,0,0,0,0
1,1993,30,11,01:30,93,93053,"48,8962100","2,4701200",10,6,...,0,0,0,1,0,0,0,0,0,0
2,1959,30,11,01:30,93,93053,"48,8962100","2,4701200",10,6,...,0,0,0,1,0,0,0,0,0,0
3,1994,30,11,02:50,93,93066,"48,9307000","2,3688000",2,3,...,0,0,0,1,0,0,0,0,0,0
4,1996,28,11,15:15,92,92036,"48,9358718","2,3191744",8,10,...,0,0,0,1,0,0,0,0,0,0


Il faut encore gérer les variables non-catégorielles : `an_naiss`, `jour`, `mois`, `hrmn`, `dep`, `com`, `lat`, `long`, `nbv`, `pr`, `pr1` et `vma`